# Introduction

Classify Urban Open Spaces from planetscope 3m imagery, with FCNN U-Net. Based on GEE FCNN notebook.

##### Version

v13
For test site.

# Libraries & imports

Authenticate and import packages.

In [ ]:
# Cloud authentication.
from google.colab import auth
auth.authenticate_user()

In [ ]:
# Import, authenticate and initialize the Earth Engine library.
import ee
ee.Authenticate()
ee.Initialize()

In [ ]:
import tensorflow as tf
print("Temsorflow version:")
print(tf.__version__)

import folium
print("Folium version:")
print(folium.__version__)

import matplotlib
import matplotlib.pyplot as plt
import os
import numpy as np

# Variables

Declare the variables that will be in use throughout the notebook.

## Specify your Cloud Storage Bucket

In [ ]:
# INSERT YOUR BUCKET HERE:
BUCKET = 'n-verde_bucket'

## Set global variables

In [ ]:
# Specify names locations for outputs.
FOLDER = '1171-UOS-256_2500-SAR'
TRAINING_BASE = 'training_patches'
EVAL_BASE = 'eval_patches'

# ---------------------- FEATURES --------------------------
# Specify inputs (Planetscope & S1 bands) to the model and the response variable.
opticalBands = ['b1', 'b2', 'b3', 'b4', 'NDVI']
sarBands = []
otherData = []  # predicted roads
BANDS = opticalBands + sarBands + otherData
RESPONSE = 'ugs'
FEATURES = BANDS + [RESPONSE]

# Specify the number of tiles the imagery will be split in
# predictions will run per tile
# will be split in SxS tiles
S = 8

# Specify the size and shape of patches expected by the model.
KERNEL_SIZE = 256
KERNEL_SHAPE = [KERNEL_SIZE, KERNEL_SIZE]
COLUMNS = [
  tf.io.FixedLenFeature(shape=KERNEL_SHAPE, dtype=tf.float32) for k in FEATURES
]
FEATURES_DICT = dict(zip(FEATURES, COLUMNS))

# ---------------------- SAMPLE SIZE --------------------------
# Sizes of the training and evaluation datasets.
TRAIN_SIZE = 2500   # 1500  # 2500  # 1200  # 16000
EVAL_SIZE = 1071    # 643   # 1071  # 320    # 4000

# Specify model training parameters.
# https://www.kite.com/python/docs/keras.backend.moving_averages.distribution_strategy_context.distribute_lib.dataset_ops.BatchDataset.shuffle
# BATCH_SIZE is dependent on your GPU memory. (e.g. on my PC it can't be larger than 4, on Colab it can be 32)
BATCH_SIZE = 16 # 4
# https://machinelearningmastery.com/how-to-stop-training-deep-neural-networks-at-the-right-time-using-early-stopping/
EPOCHS = 500
# For perfect shuffling, set the buffer size equal to the full size of the dataset.
# https://www.tensorflow.org/api_docs/python/tf/data/experimental/shuffle_and_repeat
BUFFER_SIZE = 2000

# ---------------------- OPTIMIZER --------------------------

# Optimizer with POLYNOMIAL learning rate decay
# https://keras.io/api/optimizers/learning_rate_schedules/polynomial_decay/
lr_schedule = tf.keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=0.01,   # 0.01
    decay_steps=20,    # 200
    end_learning_rate=0.0001,    # 0.00001
    power=0.5,
    cycle=False,
)
OPTIMIZER = tf.keras.optimizers.SGD(learning_rate=lr_schedule, momentum=0.9)

# ---------------------- LOSS --------------------------

# DICE LOSS
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + smooth)
def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)
LOSS = dice_coef_loss

# ---------------------- METRICS --------------------------
# https://keras.io/api/metrics/accuracy_metrics/
# METRICS = [metrics.get('binary_accuracy')]

def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + smooth)

METRICS = [tf.keras.metrics.get('binary_accuracy'),
           dice_coef,
           tf.keras.metrics.MeanIoU(num_classes=2),
           tf.keras.metrics.Recall()]

print('OK!')

# Imagery
## All

Gather and setup the imagery to use for inputs (predictors) (Planetscope mosaic, indices, SAR etc).  Display it in the notebook for a sanity check.

Prepare the response (what we want to predict). This is the Urban Open Spaces from OSM, rasterized. Display to check.

In [ ]:
# Use folium to visualize the imagery.
map = folium.Map(location=[37.981892554434936, 23.7269115882649])


# ------------------------------------------
# planetscope mosiac image of attica
planetscope = ee.Image('users/n-verde/PhD_1171/attica_feather_mask_buff400')

mapid = planetscope.getMapId({'bands': ['b4', 'b3', 'b2'], 'min': 730, 'max': 3500})
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='planetscope',
  ).add_to(map)

# ------------------------------------------
# ugs ground truth
gt = ee.Image('users/n-verde/PhD_1171/osm_open_areas_EDIT_v2_3m')
gt = gt.rename('ugs')

mapid = gt.getMapId({'bands': ['ugs'], 'min': 0, 'max': 1})
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='ground_truth',
  ).add_to(map)

# ------------------------------------------
# S1 imagery
s1_col = ee.ImageCollection("COPERNICUS/S1_GRD_FLOAT")

# get the s1 image closest to the date of the planetscope image
# get only descending orbit
s1_col = s1_col.filterDate('2020-08-27', '2020-09-02')
s1_col = s1_col.filterMetadata('instrumentMode', 'equals', 'IW')
s1_col = s1_col.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
s1_col = s1_col.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
s1_col = s1_col.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
s1_im = ee.Image(s1_col.mean()).mask(planetscope.select('b1')).select('VV', 'VH')

mapid = s1_im.getMapId({'bands': ['VV'], 'min': 0, 'max': 3.5})
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='s1_im',
  ).add_to(map)

map.add_child(folium.LayerControl())

map

## SAR Processing

### Transformations

In [ ]:
############################################################
# This function calculates the VVH indicator.
# Used for better capturing building height from Sentinel-1 at 500m spatial resolution
# (Li et al., 2020)

def addVVH(image):
  vvh = None
  vvh = image.expression('VV * (gamma ** VH)',{
          'VV': image.select('VV'),
          'VH': image.select('VH'),
          'gamma': ee.Number(5)
        })
  return image.addBands(vvh.rename('VVH'))

s1_im = addVVH(s1_im)

############################################################
# This function calculates building height.
# Used for Sentinel-1 at 500m spatial resolution
# (Li et al., 2020)

def buildingHeightS1(image):
  h = None
  h = image.expression('e ** (a * (VVH ** b) + c)',{
          'VVH': image.select('VVH'),
          'e': ee.Number(2.71828),
          'a': ee.Number(-23.61),
          'b': ee.Number(-0.06),
          'c': ee.Number(26.10),
        })
  return image.addBands(h.rename('buildingH_Natural'))

s1_im = buildingHeightS1(s1_im)

print("Calculated building height index")

# print(s1_im.getInfo())

### Turn to dB
Turn SAR bands from natural values to dB.

In [ ]:
############################################################
# This function converts to dB from Natural
def toDB(img):
    return ee.Image(img).log10().multiply(10.0)

############################################################
# This function cinverts from dB to Natural
def toNatural(img):
    return ee.Image(10.0).pow(img.divide(10.0));

# this image is used for building heights
buildingH_db = toDB(s1_im.select('buildingH_Natural'))
s1_im_dB = s1_im.addBands(buildingH_db.rename('buildingH'))

print("Converted SAR imagery from natural to dB")
# print(s1_im_dB.getInfo())

### Normalization

Find the min and max of the S1 image for rescaling to [0-1]. When normalizing data to [0-1] the training of the NN is made easier.
If you are running this section for the first time, uncomment the reducers and comment-out the `min=ee.Number(...)` and `max=ee.Number(...)`

In [ ]:
# define the study area (area of planetscope image)
aoi = ee.FeatureCollection('users/n-verde/PhD_1171/Athens_buff400')

print("finding min and max of SAR S1 bands, in order to normalize them...")
print("using percentiles instead of absolute min and max...")
print("----------")

def findMinSAR(imgBand, bandName, min_prec):
    MIN = ee.Number(imgBand.reduceRegion(**{
    'reducer': ee.Reducer.percentile([min_prec]),
    'geometry': aoi,
    'scale': 10,
    'tileScale': 4,
    # 'bestEffort': true,
    'maxPixels': 1e9
    }).get(bandName))

    return MIN

def findMaxSAR(imgBand, bandName, max_perc):
    MAX = ee.Number(imgBand.reduceRegion(**{
    'reducer': ee.Reducer.percentile([max_perc]),
    'geometry': aoi,
    'scale': 10,
    'tileScale': 4,
    # 'bestEffort': true,
    'maxPixels': 1e9
    }).get(bandName))

    return MAX

# VV ----------

bandName = "VV"
imgBand = s1_im_dB.select(bandName)

# 15.85 to 84.15 is stretch to 1σ
# 2.3 to 97.7 is stretch to 2σ
# 0.15 to 99.85 is stretch to 3σ

# MIN = ee.Number(findMinSAR(imgBand, bandName, 0.15))
MIN = ee.Number(0.35441243231694636)

# MAX = ee.Number(findMaxSAR(imgBand, bandName, 99.85))
MAX = ee.Number(0.3544124323169465)

print('old min', MIN.getInfo())
print('old max', MAX.getInfo())

test = ee.Algorithms.If(MIN.lt(ee.Number(0)), 1, 0)

if (test.getInfo()==1):
    imgBand = imgBand.add(MIN.abs()) # turn negative values to possitive
    MAX = MAX.add(MIN.abs()) # change new max
    MIN = ee.Number(0) # change min to 0

min_VV = MIN
max_VV = MAX

print("min of VV:", min_VV.getInfo())
print("max of VV:", max_VV.getInfo())

# VH ----------

bandName = "VH"
imgBand = s1_im_dB.select(bandName)

# MIN = findMinSAR(imgBand, bandName, 0.15)
MIN = ee.Number(0.05221765024397127)

# MAX = findMaxSAR(imgBand, bandName, 99.85)
MAX = ee.Number(1.3602214673473254)

print('old min', MIN.getInfo())
print('old max', MAX.getInfo())

test = ee.Algorithms.If(MIN.lt(ee.Number(0)), ee.Number(1), ee.Number(0))

if (test.getInfo()==1):
    imgBand = imgBand.add(MIN.abs()) # turn negative values to possitive
    MAX = MAX.add(MIN.abs()) # change new max
    MIN = ee.Number(0) # change min to 0

min_VH = MIN
max_VH = MAX

print("min of VH:", min_VH.getInfo())
print("max of VH:", max_VH.getInfo())

# buildingH ----------

bandName = "buildingH"
imgBand = s1_im_dB.select(bandName)

# MIN = findMinSAR(imgBand, bandName, 0.15)
MIN = ee.Number(-34.476049329083615)

# MAX = findMaxSAR(imgBand, bandName, 99.85)
MAX = ee.Number(31.480003022441355)

print('old min', MIN.getInfo())
print('old max', MAX.getInfo())

test = ee.Algorithms.If(MIN.lt(ee.Number(0)), 1, 0)

if (test.getInfo()==1):
    imgBand = imgBand.add(MIN.abs()) # turn negative values to possitive
    MAX = MAX.add(MIN.abs()) # change new max
    MIN = ee.Number(0) # change min to 0

min_buildingH = MIN
max_buildingH = MAX

print("min of buildingH:", min_buildingH.getInfo())
print("max of buildingH:", max_buildingH.getInfo())

print("----------")
print("done!")

Rescale to [0-1] (normalize)

In [ ]:
def rescaleFixedMinMax(img, bandName, MIN, MAX):
    Min = ee.Number(MIN)
    Max = ee.Number(MAX)

    imgBand = img.select(bandName)

    imgBand = imgBand.float()

    imgBandNorm = imgBand.divide(Max) # normalize the data to 0 - 1

    return imgBandNorm

# Apply the rescale function ----------

# VV ----------
VV_res = rescaleFixedMinMax(s1_im_dB, "VV", min_VV, max_VV)

# VH ----------
VH_res = rescaleFixedMinMax(s1_im_dB, "VH", min_VH, max_VH)

# buildingH -----------
buildingH_res = rescaleFixedMinMax(s1_im_dB,"buildingH", min_buildingH, max_buildingH)

print("done rescaling SAR imagery to 0-1")

## Optical Processing

### Transformations

In [ ]:
#############################################################
# This function calculates NDVI from a S2 image, and adds it to the image stack.
def addNDVI(image):
    return image.addBands(image.normalizedDifference(['b4', 'b3']).rename('NDVI')) # change new band name

planetscope = addNDVI(planetscope)

print("Calculated NDVI")


### Normalization

In [ ]:
# define the study area (area of planetscope image)
aoi = ee.FeatureCollection('users/n-verde/PhD_1171/Athens_buff400')

print("finding min and max of planetscope bands, in order to normalize them...")
print("----------")

def findMin(imgBand, bandName):
    MIN = ee.Number(imgBand.reduceRegion(**{
    'reducer': ee.Reducer.min(),
    'geometry': aoi,
    'scale': 9,
    'tileScale': 4,
    # 'bestEffort': true,
    'maxPixels': 1e9
    }).get(bandName))

    return MIN

def findMax(imgBand, bandName):
    MAX = ee.Number(imgBand.reduceRegion(**{
    'reducer': ee.Reducer.max(),
    'geometry': aoi,
    'scale': 9,
    'tileScale': 4,
    # 'bestEffort': true,
    'maxPixels': 1e9
    }).get(bandName))

    return MAX

# b1 ----------

bandName = "b1"
imgBand = planetscope.select(bandName)

# MIN = findMin(imgBand, bandName)
MIN = ee.Number(50)

# MAX = findMax(imgBand, bandName)
MAX = ee.Number(8233)

print('old min', MIN.getInfo())
print('old max', MAX.getInfo())

test = ee.Algorithms.If(MIN.lt(ee.Number(0)), ee.Number(1), ee.Number(0))

if (test.getInfo()==1):
    imgBand = imgBand.add(MIN.abs()) # turn negative values to possitive
    MAX = MAX.add(MIN.abs()) # change new max
    MIN = ee.Number(0) # change min to 0

min_b1 = MIN
max_b1 = MAX

print("min of b1:", min_b1.getInfo())
print("max of b1:", max_b1.getInfo())

# b2 ----------

bandName = "b2"
imgBand = planetscope.select(bandName)

# MIN = findMin(imgBand, bandName)
MIN = ee.Number(251)

# MAX = findMax(imgBand, bandName)
MAX = ee.Number(9164)

print('old min', MIN.getInfo())
print('old max', MAX.getInfo())

test = ee.Algorithms.If(MIN.lt(ee.Number(0)), ee.Number(1), ee.Number(0))

if (test.getInfo()==1):
    imgBand = imgBand.add(MIN.abs()) # turn negative values to possitive
    MAX = MAX.add(MIN.abs()) # change new max
    MIN = ee.Number(0) # change min to 0

min_b2 = MIN
max_b2 = MAX

print("min of b2:", min_b2.getInfo())
print("max of b2:", max_b2.getInfo())

# b3 ----------

bandName = "b3"
imgBand = planetscope.select(bandName)

# MIN = findMin(imgBand, bandName)
MIN = ee.Number(159)

# MAX = findMax(imgBand, bandName)
MAX = ee.Number(11765)

print('old min', MIN.getInfo())
print('old max', MAX.getInfo())

test = ee.Algorithms.If(MIN.lt(ee.Number(0)), ee.Number(1), ee.Number(0))

if (test.getInfo()==1):
    imgBand = imgBand.add(MIN.abs()) # turn negative values to possitive
    MAX = MAX.add(MIN.abs()) # change new max
    MIN = ee.Number(0) # change min to 0

min_b3 = MIN
max_b3 = MAX

print("min of b3:", min_b3.getInfo())
print("max of b3:", max_b3.getInfo())

# b4 ----------

bandName = "b4"
imgBand = planetscope.select(bandName)

# MIN = findMin(imgBand, bandName)
MIN = ee.Number(1)

# MAX = findMax(imgBand, bandName)
MAX = ee.Number(12127)

print('old min', MIN.getInfo())
print('old max', MAX.getInfo())

test = ee.Algorithms.If(MIN.lt(ee.Number(0)), ee.Number(1), ee.Number(0))

if (test.getInfo()==1):
    imgBand = imgBand.add(MIN.abs()) # turn negative values to possitive
    MAX = MAX.add(MIN.abs()) # change new max
    MIN = ee.Number(0) # change min to 0

min_b4 = MIN
max_b4 = MAX

print("min of b4:", min_b4.getInfo())
print("max of b4:", max_b4.getInfo())

# NDVI ----------

bandName = "NDVI"
imgBand = planetscope.select(bandName)

# MIN = findMin(imgBand, bandName)
MIN = ee.Number(-0.9998300186979432)

# MAX = findMax(imgBand, bandName)
MAX = ee.Number(0.8263786957433424)

print('old min', MIN.getInfo())
print('old max', MAX.getInfo())

test = ee.Algorithms.If(MIN.lt(ee.Number(0)), ee.Number(1), ee.Number(0))

if (test.getInfo()==1):
    imgBand = imgBand.add(MIN.abs()) # turn negative values to possitive
    MAX = MAX.add(MIN.abs()) # change new max
    MIN = ee.Number(0) # change min to 0

min_NDVI = MIN
max_NDVI = MAX

print("min of NDVI:", min_NDVI.getInfo())
print("max of NDVI:", max_NDVI.getInfo())

print("----------")
print("done!")

Rescale to [0-1] (normalize)

In [ ]:
def rescaleFixedMinMax(img, bandName, MIN, MAX):
    Min = ee.Number(MIN)
    Max = ee.Number(MAX)

    imgBand = img.select(bandName)

    imgBand = imgBand.float()

    imgBandNorm = imgBand.divide(Max) # normalize the data to 0 - 1

    return imgBandNorm

# Apply the rescale function to all bands of the planetscope image
# b1 ----------
b1_res = rescaleFixedMinMax(planetscope,"b1", min_b1, max_b1)

# b2 ----------
b2_res = rescaleFixedMinMax(planetscope,"b2", min_b2, max_b2)

# b3 ----------
b3_res = rescaleFixedMinMax(planetscope,"b3", min_b3, max_b3)

# b4 ----------
b4_res = rescaleFixedMinMax(planetscope,"b4", min_b4, max_b4)

# NDVI ----------
NDVI_res = rescaleFixedMinMax(planetscope,"NDVI", min_NDVI, max_NDVI)

print("done rescaling optical imagery to 0-1")

## Stack

In [ ]:
# merge the bands to a single image
image_res = b1_res.addBands(b2_res).addBands(b3_res).addBands(b4_res).addBands(NDVI_res)
image_res = image_res.addBands(VV_res).addBands(VH_res).addBands(buildingH_res)

# display rescaled image
mapid = image_res.getMapId({'bands': ['NDVI'], 'min': 0, 'max': 1})
map = folium.Map(location=[37.981892554434936, 23.7269115882649])
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='rescaled',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

## Masking

Mask the rescaled planetscope image to the samples extent (boxes image).

In [ ]:
boxes = ee.Image('users/n-verde/PhD_1171/BOXES_512x512_TRAIN_VAL')
masked_image = image_res.mask(boxes)

# display the masked image
mapid = masked_image.getMapId({'bands': ['b4', 'b3', 'b2'], 'min': 0, 'max': 0.4})
map = folium.Map(location=[37.981892554434936, 23.7269115882649])
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='planetscope_masked',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

## Stack image with response

Stack the 2D images (rescaled planetscope image and road samples) to create a single image from which samples can be taken. Convert the image into an array image in which each pixel stores 256x256 patches of pixels for each band.  This is a key step that bears emphasis: to export training patches, convert a multi-band image to [an array image](https://developers.google.com/earth-engine/arrays_array_images#array-images) using [`neighborhoodToArray()`](https://developers.google.com/earth-engine/api_docs#eeimageneighborhoodtoarray), then sample the image at points.

In [ ]:
featureStack = ee.Image.cat([
  masked_image.select(BANDS),
  gt.select(RESPONSE)
]).float()

print('features for model: ', featureStack.bandNames().getInfo())

list = ee.List.repeat(1, KERNEL_SIZE)
lists = ee.List.repeat(list, KERNEL_SIZE)
kernel = ee.Kernel.fixed(KERNEL_SIZE, KERNEL_SIZE, lists)

arrays = featureStack.neighborhoodToArray(kernel)

# Split image to tiles
Define a function for displaying Earth Engine image tiles on a folium map

In [ ]:
# Define a method for displaying Earth Engine image tiles on a folium map.
def add_ee_layer(self, ee_object, vis_params, name):

    try:
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)

    except:
        print("Could not display {}".format(name))

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [ ]:
# This function splits a geometry into equal-sized subrectangles.
# splits a geometry to parts^2 subregions
# Assumes that the geometry has only 4 vertices.
# returns a list of polygons (you can map over it)
def split(geom, nSplits):

  one = ee.Number(1)

  # Return (nSplit+1)^2 coordinate pairs, given 4 vertices.
  def toPts(rect, nSplits):
    k = ee.List.sequence(0, None, one.divide(nSplits), one.add(nSplits));

    def f1(x):

      def f2(y):
        xp = one.subtract(x)
        yp = one.subtract(y)
        coeffs = ee.Array([[xp.multiply(yp), yp.multiply(x), xp.multiply(y), ee.Number(x).multiply(y)]])
        return coeffs.matrixMultiply(rect).project([1])

      return k.map(f2)

    return k.map(f1).flatten()

  # Return nSplit^2 polygons, given the list of vertices built by toPts().
  def toRects(pts, nSplits):
    offsets = ee.List([0, 1, one.add(nSplits).add(one), one.add(nSplits)])
    k1 = ee.List.sequence(0, None, one.add(nSplits), nSplits)

    def f3(i):
      k2 = ee.List.sequence(i, None, 1, nSplits)

      def f4(j):

        def f5(offset):
          return ee.Array(pts.get(ee.Number(j).add(offset))).toList()

        return ee.Geometry.Polygon(offsets.map(f5))

      return k2.map(f4)

    return k1.map(f3).flatten()


  # Get the 4 vertices.  Assumes that the scene geometry has only 4 vertices.
  rect = ee.List(geom.coordinates().get(0))
  rect = ee.Array([rect.get(0), rect.get(1), rect.get(3), rect.get(2)])

  pts = toPts(rect, nSplits)
  rects = toRects(pts, nSplits)

  print('Parts that geometry was splitted to ', rects.getInfo())

  return ee.List(rects)

# get planetscope image bounds in a box
i = planetscope.geometry().bounds().transform('EPSG:4326',100);

# split to SxS tiles
splitted = split(i,S);


In [ ]:
# show on Map

no =29
spl = ee.Geometry(ee.List(splitted.get(no-1)))
desc = 'tile' + str(int(no-1)) + 'from the split'

map = folium.Map(location=[37.981892554434936, 23.7269115882649])

# planetscope mosiac image of attica
image = planetscope

mapid = image.getMapId({'bands': ['b4', 'b3', 'b2'], 'min': 730, 'max': 3500})
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='planetscope',
  ).add_to(map)

# planetscope bounds
map.add_ee_layer(i,{}, 'planetscope bounds')

# tile
map.add_ee_layer(spl, {}, desc)

map.add_child(folium.LayerControl())
map

# Sampling

Use some pre-made geometries to sample the stack in strategic locations.  Specifically, these are hand-made polygons in which to take the 256x256 samples.  Display the sampling polygons on a map, red for training polygons, blue for evaluation.

In [ ]:
trainingPolys = ee.FeatureCollection('users/n-verde/PhD_1171/UOS_grid_512_TRAINING')
evalPolys = ee.FeatureCollection('users/n-verde/PhD_1171/UOS_grid_512_VALIDATION')

polyImage = ee.Image(0).byte().paint(trainingPolys, 1).paint(evalPolys, 2)
polyImage = polyImage.updateMask(polyImage)

# display the sample boxes
mapid = polyImage.getMapId({'min': 1, 'max': 2, 'palette': ['red', 'blue']})
map = folium.Map(location=[37.981892554434936, 23.7269115882649])
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='training & validation polygons',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

Take samples from each polygon and merge the results into a single export.  The key step is sampling the array image at points, to get all the pixels in a 256x256 neighborhood at each point.  It's worth noting that to build the training and testing data, you export a single TFRecord file that contains patches of pixel values in each record.  You do NOT need to export each training/testing patch to a different image.  Since each record potentially contains a lot of data (especially with big patches or many input bands), some manual sharding of the computation is necessary to avoid the `computed value too large` error.  Specifically, the following code takes multiple (smaller) samples within each geometry, merging the results to get a single export.
For whole Attica takes around **1h** in GEE.

In [ ]:
# Convert the feature collections to lists for iteration.
trainingPolysList = trainingPolys.toList(trainingPolys.size())
evalPolysList = evalPolys.toList(evalPolys.size())

# These numbers determined experimentally.
n = 10 # Number of shards in each polygon.
N =  100 # Total sample size in each polygon.

# Export all the training data (in many pieces), with one task
# per geometry.
for g in range(trainingPolys.size().getInfo()):
  geomSample = ee.FeatureCollection([])
  for i in range(n):
    sample = arrays.sample(
      region = ee.Feature(trainingPolysList.get(g)).geometry(),
      scale = 1,
      numPixels = N / n, # Size of the shard.
      seed = i,
      tileScale = 16
    )
    geomSample = geomSample.merge(sample)

  desc = TRAINING_BASE + '_g' + str(g)
  task = ee.batch.Export.table.toCloudStorage(
    collection = geomSample,
    description = desc,
    bucket = BUCKET,
    fileNamePrefix = FOLDER + '/' + desc,
    fileFormat = 'TFRecord',
    selectors = BANDS + [RESPONSE]
  )
  task.start()

# Export all the evaluation data.
for g in range(evalPolys.size().getInfo()):
  geomSample = ee.FeatureCollection([])
  for i in range(n):
    sample = arrays.sample(
      region = ee.Feature(evalPolysList.get(g)).geometry(),
      scale = 1,
      numPixels = N / n,
      seed = i,
      tileScale = 16
    )
    geomSample = geomSample.merge(sample)

  desc = EVAL_BASE + '_g' + str(g)
  task = ee.batch.Export.table.toCloudStorage(
    collection = geomSample,
    description = desc,
    bucket = BUCKET,
    fileNamePrefix = FOLDER + '/' + desc,
    fileFormat = 'TFRecord',
    selectors = BANDS + [RESPONSE]
  )
  task.start()

## Training data

Load the data exported from Earth Engine to Cloud Storage into a `tf.data.Dataset`.  The following are helper functions for that.

In [ ]:
def parse_tfrecord(example_proto):
  """The parsing function.
  Read a serialized example into the structure defined by FEATURES_DICT.
  Args:
    example_proto: a serialized Example.
  Returns:
    A dictionary of tensors, keyed by feature name.
  """
  return tf.io.parse_single_example(example_proto, FEATURES_DICT)


def to_tuple(inputs):
  """Function to convert a dictionary of tensors to a tuple of (inputs, outputs).
  Turn the tensors returned by parse_tfrecord into a stack in HWC shape.
  Args:
    inputs: A dictionary of tensors, keyed by feature name.
  Returns:
    A tuple of (inputs, outputs).
  """
  inputsList = [inputs.get(key) for key in FEATURES]
  stacked = tf.stack(inputsList, axis=0)
  # Convert from CHW to HWC
  stacked = tf.transpose(stacked, [1, 2, 0])
  return stacked[:,:,:len(BANDS)], stacked[:,:,len(BANDS):]


def get_dataset(pattern):
  """Function to read, parse and format to tuple a set of input tfrecord files.
  Get all the files matching the pattern, parse and convert to tuple.
  Args:
    pattern: A file pattern to match in a Cloud Storage bucket.
  Returns:
    A tf.data.Dataset
  """
  glob = tf.io.gfile.glob(pattern)
  dataset = tf.data.TFRecordDataset(glob, compression_type='GZIP')
  dataset = dataset.map(parse_tfrecord, num_parallel_calls=5)
  dataset = dataset.map(to_tuple, num_parallel_calls=5)
  return dataset

Use the helpers to read in the training dataset.  Print the first record to check.

In [ ]:
def get_training_dataset():
	"""Get the preprocessed training dataset
  Returns:
    A tf.data.Dataset of training data.
  """
	glob = 'gs://' + BUCKET + '/' + FOLDER + '/' + TRAINING_BASE + '*'
	dataset = get_dataset(glob)
    # https://stackoverflow.com/questions/53514495/what-does-batch-repeat-and-shuffle-do-with-tensorflow-dataset
    # https://www.tensorflow.org/api_docs/python/tf/data/experimental/shuffle_and_repeat
	dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
	return dataset

training = get_training_dataset()

print(iter(training.take(1)).next())

## Evaluation data

Now do the same thing to get an evaluation dataset.  Note that unlike the training dataset, the evaluation dataset has a batch size of 1, is not repeated and is not shuffled.

In [ ]:
def get_eval_dataset():
	"""Get the preprocessed evaluation dataset
  Returns:
    A tf.data.Dataset of evaluation data.
  """
	glob = 'gs://' + BUCKET + '/' + FOLDER + '/' + EVAL_BASE + '*'
	dataset = get_dataset(glob)
	dataset = dataset.batch(1).repeat()
	return dataset

evaluation = get_eval_dataset()

print(iter(training.take(1)).next())

## Turn tfrecord samples to images
If you want to see how the samples look like, turn them from tfrecords to images (Only works on Jupyter when run localy).

In [ ]:
# import glob
# import numpy as np

# input_path = 'gs://' + BUCKET + '/' + FOLDER + '/' + TRAINING_BASE + '/'
# output_path = 'gs://' + BUCKET + '/' + FOLDER + '/' + TRAINING_BASE + '/' + 'images'


# files = glob.glob(input_path + '/*.tfrecord.gz')
# if len(files) == 0:
#     print('invalid input path : ' + input_path)
#     quit(-1)
# print('input files  : {}'.format(input_path + '/*.tfrecord.gz'))

# if not os.path.exists(output_path):
#     os.mkdir(output_path)
#     print('output dir : {} (auto created)'.format(output_path))
# else:
#     print('output dir : {} '.format(output_path))

# num = 0

# for file in files:

#     dataset = tf.data.TFRecordDataset(file, compression_type='GZIP')

#     parsedDataset = dataset.map(lambda example: tf.io.parse_single_example(example, FEATURES_DICT))

#     for image_features in parsedDataset:
#         image_NDVI = image_features['NDVI'].numpy()
#         image_ugs = image_features['ugs'].numpy()
#         image_NIR = image_features['b4'].numpy()
#         image_R = image_features['b3'].numpy()
#         image_G = image_features['b2'].numpy()
#         image_B = image_features['b1'].numpy()
#         image_RGB = np.dstack((image_NIR, image_R, image_ugs))

#         # write the image to file
#         matplotlib.image.imsave(output_path + '/' + 'patch' + str(num)  + '.png', image_RGB)

#         num = num + 1


# Model

Here we use the Keras implementation of the U-Net model.  The U-Net model takes 256x256 pixel patches as input and outputs per-pixel class labels output.  We can implement the model essentially unmodified. Since Urban Open Spaces (UOS) are representnted in a binary way (0=non-UOS, 1=UOS], a sigmoid activation function is suitable here.

In [ ]:
def conv_block(input_tensor, num_filters):
	encoder = tf.keras.layers.Conv2D(num_filters, (3, 3), padding='same')(input_tensor)
	encoder = tf.keras.layers.BatchNormalization()(encoder)
	encoder = tf.keras.layers.Activation('relu')(encoder)
	encoder = tf.keras.layers.Conv2D(num_filters, (3, 3), padding='same')(encoder)
	encoder = tf.keras.layers.BatchNormalization()(encoder)
	encoder = tf.keras.layers.Activation('relu')(encoder)
	return encoder

def encoder_block(input_tensor, num_filters):
	encoder = conv_block(input_tensor, num_filters)
	encoder_pool = tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2))(encoder)
	return encoder_pool, encoder

def decoder_block(input_tensor, concat_tensor, num_filters):
	decoder = tf.keras.layers.Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding='same')(input_tensor)
	decoder = tf.keras.layers.concatenate([concat_tensor, decoder], axis=-1)
	decoder = tf.keras.layers.BatchNormalization()(decoder)
	decoder = tf.keras.layers.Activation('relu')(decoder)
	decoder = tf.keras.layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
	decoder = tf.keras.layers.BatchNormalization()(decoder)
	decoder = tf.keras.layers.Activation('relu')(decoder)
	decoder = tf.keras.layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
	decoder = tf.keras.layers.BatchNormalization()(decoder)
	decoder = tf.keras.layers.Activation('relu')(decoder)
	return decoder

def get_model():
	inputs = tf.keras.layers.Input(shape=[None, None, len(BANDS)]) # 256
	encoder0_pool, encoder0 = encoder_block(inputs, 32) # 128
	encoder1_pool, encoder1 = encoder_block(encoder0_pool, 64) # 64
	encoder2_pool, encoder2 = encoder_block(encoder1_pool, 128) # 32
	encoder3_pool, encoder3 = encoder_block(encoder2_pool, 256) # 16
	encoder4_pool, encoder4 = encoder_block(encoder3_pool, 512) # 8
	center = conv_block(encoder4_pool, 1024) # center
	decoder4 = decoder_block(center, encoder4, 512) # 16
	decoder3 = decoder_block(decoder4, encoder3, 256) # 32
	decoder2 = decoder_block(decoder3, encoder2, 128) # 64
	decoder1 = decoder_block(decoder2, encoder1, 64) # 128
	decoder0 = decoder_block(decoder1, encoder0, 32) # 256
	outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(decoder0)
    # https://machinelearningmastery.com/choose-an-activation-function-for-deep-learning/
    # https://www.quora.com/Does-it-make-sense-to-use-Relu-activation-on-the-output-neuron-for-binary-classification-If-not-why?share=1

	model = tf.keras.Model(inputs=[inputs], outputs=[outputs])

	model.compile(
		optimizer=OPTIMIZER,
		loss=LOSS,
		metrics=METRICS)

	return model

# Training

## Train
You train a Keras model by calling `.fit()` on it.  Here we're going to train for 500 epochs (Takes around **4.5h** with a GPU on colab).


In [ ]:
m = get_model()

# path for best model
bpath = 'gs://' + BUCKET + '/' + FOLDER + '/' + 'best_model-epoch{epoch:02d}-loss{loss:.3f}-dice_coef{dice_coef:.3f}'

# best model is saved in every checkpoint
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=bpath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)


# history is a dictionary holding loss & accuracy at each epoch
history = m.fit(
    x=training,
    epochs=EPOCHS,
    steps_per_epoch=int(TRAIN_SIZE / BATCH_SIZE),
    validation_data=evaluation,
    validation_steps=EVAL_SIZE,
    # for verbose see https://github.com/tensorflow/tensorflow/issues/37876
    verbose=1,
    callbacks=[model_checkpoint_callback]
)

# For training loss, keras does a running average over the batches. For validation loss, a conventional average over
# all the batches in validation data is performed. The training accuracy is the average of the accuracy values for each
# batch of training data during training. (https://github.com/keras-team/keras/issues/10426)

# save history to file
hpath = 'gs://' + BUCKET + '/' + FOLDER + '/' + 'MODEL_HISTORY.npy'
np.save(hpath,history.history)

# get the latest (best) model saved
path = 'gs://' + BUCKET + '/' + FOLDER

# path of best model
latest = tf.train.latest_checkpoint(path)
m.load_weights(latest)

m.summary()


##Load a model and continue training

In [ ]:
EPOCHS_LEFT = 53

# load the checkpoint from disk
print("loading model...")

m = get_model()

# get the latest (best) model saved
path = 'gs://' + BUCKET + '/' + FOLDER

# path of best model
latest = tf.train.latest_checkpoint(path)

m.load_weights(latest)

# loss,acc = m.evaluate(evaluation, verbose=2)
# print("Restored model, accuracy: {:5.2f}%".format(100*acc))

# path for best model
bpath = 'gs://' + BUCKET + '/' + FOLDER + '/' + 'best_model-epoch{epoch:02d}-loss{loss:.3f}-dice_coef{dice_coef:.3f}'

# best model is saved in every checkpoint
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=bpath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

# resume the training where we left off
history2 = m.fit(
  x=training,
  epochs=EPOCHS_LEFT,
  steps_per_epoch=int(TRAIN_SIZE / BATCH_SIZE),
  validation_data=evaluation,
  validation_steps=EVAL_SIZE,
  # for verbose see https://github.com/tensorflow/tensorflow/issues/37876
  verbose=1,
  callbacks=[model_checkpoint_callback]
)


# save history to file
hpath = 'gs://' + BUCKET + '/' + FOLDER + '/' + 'MODEL_HISTORY.npy'
np.save(hpath,history2.history)

# get the latest (best) model saved
path = 'gs://' + BUCKET + '/' + FOLDER

# path of best model
latest = tf.train.latest_checkpoint(path)
m.load_weights(latest)

m.summary()

## Accuracy
plot the train and validation loss as well as the train and validation accuracy

In [ ]:
# # load history from file
# hpath = 'gs://' + BUCKET + '/' + FOLDER + '/' + 'MODEL_HISTORY.npy'
# history=np.load(hpath,allow_pickle='TRUE').item()

# Plotting both losses simultaneously
l = plt.plot(history.history['loss'], color='gray', label='dice loss')

l = plt.title('model loss')
l = plt.xlabel('epoch')
l = plt.legend(loc='upper left')

# save to file
plt.savefig(os.path.join('gs://' + BUCKET + '/' + FOLDER + '/' +  'loss.jpg'), dpi=300)

l = plt.show()

# Plotting both accuracy simultaneously
a = plt.plot(history.history['recall'], color='blue', label='recall')
a = plt.plot(history.history['binary_accuracy'], color='orange', label='binary_accuracy')
a = plt.plot(history.history['mean_io_u'], color='red', label='IoU')
a = plt.plot(history.history['dice_coef'], color='gray', label='dice coeficient')

a = plt.title('model accuracy')
a = plt.ylabel('accuracy')
a = plt.xlabel('epoch')
a = plt.legend(loc='lower right')

# save to file
plt.savefig(os.path.join('gs://' + BUCKET + '/' + FOLDER + '/' + 'accuracy.jpg'), dpi=300)

a = plt.show()


# Prediction

The prediction pipeline is:

1.  Export imagery on which to do predictions from Earth Engine in TFRecord format to a Cloud Storge bucket.
2.  Use the trained model to make the predictions.
3.  Write the predictions to a TFRecord file in a Cloud Storage.
4.  Upload the predictions TFRecord file to Earth Engine.

The following functions handle this process.  It's useful to separate the export from the predictions so that you can experiment with different models without running the export every time.

Export and prediction is done by tile.

Now there's all the code needed to run the prediction pipeline, all that remains is to specify the output region in which to do the prediction, the names of the output files, where to put them, and the shape of the outputs.  In terms of the shape, the model is trained on 256x256 patches, but can work (in theory) on any patch that's big enough with even dimensions ([reference](https://www.cv-foundation.org/openaccess/content_cvpr_2015/papers/Long_Fully_Convolutional_Networks_2015_CVPR_paper.pdf)).  Because of tile boundary artifacts, give the model slightly larger patches for prediction, then clip out the middle 256x256 patch.  This is controlled with a kernel buffer, half the size of which will extend beyond the kernel buffer.  For example, specifying a 128x128 kernel will append 64 pixels on each side of the patch, to ensure that the pixels in the output are taken from inputs completely covered by the kernel.

In [ ]:
# Output assets folder: YOUR FOLDER
user_folder = 'users/n-verde' # INSERT YOUR GEE FOLDER HERE.

# Base file name to use for TFRecord files and assets.
attica_image_base = '1171_UOS_'
# Half this will extend on the sides of each patch.
attica_kernel_buffer = [128, 128]


## Export images

In [ ]:
def doExport(out_image_base, kernel_buffer, region, tileNum):
  """Run the image export task.  Block until complete.
  """
  task = ee.batch.Export.image.toCloudStorage(
    image = image_res.select(BANDS),
    description = out_image_base + str(tileNum) + '_',
    bucket = BUCKET,
    fileNamePrefix = FOLDER + '/' + out_image_base + str(tileNum) + '_',
    region = region.getInfo()['coordinates'],
    scale = 1,
    fileFormat = 'TFRecord',
    maxPixels = 1e10,
    formatOptions = {
      'patchDimensions': KERNEL_SHAPE,
      'kernelSize': kernel_buffer,
      'compressed': True,
      'maxFileSize': 104857600
    }
  )
  task.start()

#   # Block until the task completes.
#   print('Running image export to Google Drive...')
#   import time
#   while task.active():
#     time.sleep(30)

#   # Error condition
#   if task.status()['state'] != 'COMPLETED':
#     print('Error with image export.')
#   else:
#     print('Image export completed.')

Run the export (all attica by tile, it takes **2-3h** on GEE).

In [ ]:
# Run the export for tiles of Attica

# Attica in tiles

tilesList = [36] # if you want to run for specific tiles

# first tile
tile_number = int(1)

# loop
for i in range(1,(len(splitted.getInfo())+1)):
# for i in tilesList: # if you want to run for specific tiles

  # # option a. without buffer
  # tile = splitted.getInfo()[i-1]
  # region = tile["coordinates"]
  # # print(region)

  # option b. with buffer
  # buffer the tile 250m to avoid data gaps between tiles
  # the distance of 250m was found empirically by running the code and seeing the gaps
  prj = ee.Projection('EPSG:3035');  # European projection
  buffer = ee.Geometry(ee.List(splitted.get(i-1))).buffer(250,None,prj).bounds(0.1)
  buffered_tile = buffer.getInfo()
  buffered_region = buffered_tile["coordinates"]
  region = buffered_region
  # print(region)

  tile_region = ee.Geometry.Polygon(region, None, False)

  # convert "1" to "01"
  tile_number_string = str(tile_number).zfill(2)

  doExport(attica_image_base, attica_kernel_buffer, tile_region, tile_number_string)

  print('done exporting for tile: ', tile_number_string, region)
  tile_number = tile_number + 1

  print('OK!')

## Load a saved model

The following code loads the best epoch for the model trained above, which you
can use for predictions right away. Loading model takes around **15-40m**.

Model at **Epoch 461**

Accuracy:
`loss: -0.5551 - binary_accuracy: 0.9444 - dice_coef: 0.5551 - mean_io_u: 0.4755 - recall: 0.5634`
`Total params: 31,127,361 - Trainable params: 31,111,361 - Non-trainable params: 16,000`



In [ ]:
m = get_model()

# get the latest (best) model saved
path = 'gs://' + BUCKET + '/' + FOLDER

# path of best model
latest = tf.train.latest_checkpoint(path)
m.load_weights(latest)

m.summary()

## Prediction

Run the prediction and upload to GEE (for whole attica, *prediction* takes **20m** and *upload* to GEE takes around **... h**).

If the GEE image upload fails with the error "Cannot read mixer", try the specific tiles again, by typing their numbers in the "tilesList" variable and changing the for loop to v2.

In [ ]:
def doPrediction(out_image_base, user_folder, kernel_buffer, region):
  """Perform inference on exported imagery, upload to Earth Engine.
  """

  print('Looking for TFRecord files...')

  # Get a list of all the files in the output bucket.
  filesList = !gsutil ls 'gs://'{BUCKET}'/'{FOLDER}

  # Get only the files generated by the image export for the specific tile.
  exportFilesList = [s for s in filesList if out_image_base in s]

  # Get the list of image files and the JSON mixer file.
  imageFilesList = []
  jsonFile = None
  for f in exportFilesList:
    if f.endswith('.tfrecord.gz'):
      imageFilesList.append(f)
    elif f.endswith('.json'):
      jsonFile = f

  # Make sure the files are in the right order.
  imageFilesList.sort()

  from pprint import pprint
  pprint(imageFilesList)
  print(jsonFile)

  import json
  # Load the contents of the mixer file to a JSON object.
  jsonText = !gsutil cat {jsonFile}
  # Get a single string w/ newlines from the IPython.utils.text.SList
  mixer = json.loads(jsonText.nlstr)
  pprint(mixer)
  patches = mixer['totalPatches']

  # Get set up for prediction.
  x_buffer = int(kernel_buffer[0] / 2)
  y_buffer = int(kernel_buffer[1] / 2)

  buffered_shape = [
      KERNEL_SHAPE[0] + kernel_buffer[0],
      KERNEL_SHAPE[1] + kernel_buffer[1]]

  imageColumns = [
    tf.io.FixedLenFeature(shape=buffered_shape, dtype=tf.float32)
      for k in BANDS
  ]

  imageFeaturesDict = dict(zip(BANDS, imageColumns))

  def parse_image(example_proto):
    return tf.io.parse_single_example(example_proto, imageFeaturesDict)

  def toTupleImage(inputs):
    inputsList = [inputs.get(key) for key in BANDS]
    stacked = tf.stack(inputsList, axis=0)
    stacked = tf.transpose(stacked, [1, 2, 0])
    return stacked

   # Create a dataset from the TFRecord file(s) in Cloud Storage.
  imageDataset = tf.data.TFRecordDataset(imageFilesList, compression_type='GZIP')
  imageDataset = imageDataset.map(parse_image, num_parallel_calls=5)
  imageDataset = imageDataset.map(toTupleImage).batch(1)

  # Perform inference.
  print('Running predictions...')
  predictions = m.predict(imageDataset, steps=patches, verbose=1)
  # print(predictions[0])

  print('Writing predictions...')
  out_image_file = 'gs://' + BUCKET + '/' + FOLDER + '/' + out_image_base + '.TFRecord'
  writer = tf.io.TFRecordWriter(out_image_file)
  patches = 0
  for predictionPatch in predictions:
    print('Writing patch ' + str(patches) + '...')
    predictionPatch = predictionPatch[
        x_buffer:x_buffer+KERNEL_SIZE, y_buffer:y_buffer+KERNEL_SIZE]

    # Create an example.
    example = tf.train.Example(
      features=tf.train.Features(
        feature={
          'r': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=predictionPatch.flatten()))
        }
      )
    )
    # Write the example.
    writer.write(example.SerializeToString())
    patches += 1

  writer.close()

  # Start the upload.
  out_image_asset = user_folder + '/' + out_image_base
  !earthengine upload image --asset_id={out_image_asset} {out_image_file} {jsonFile}

print('DONE!')

In [ ]:
# Run the prediction for tiles of Attica

# just to invoke earth engine so it doesn't disconnect
ee.Authenticate()
ee.Initialize()

#####################################################
## FOR ALL TILES --> (V1)

# first tile
tile_number = int(1)

# loop
for i in range(1,(len(splitted.getInfo())+1)): # V1

  # # option a. without buffer
  # tile = splitted.getInfo()[i-1]
  # region = tile["coordinates"]
  # # print(region)

  # option b. with buffer
  # buffer the tile 250m to avoid data gaps between tiles
  # the distance of 250m was found empirically by running the code and seeing the gaps
  prj = ee.Projection('EPSG:3035');  # European projection
  buffer = ee.Geometry(ee.List(splitted.get(i-1))).buffer(250,None,prj).bounds(0.1)
  buffered_tile = buffer.getInfo()
  buffered_region = buffered_tile["coordinates"]
  region = buffered_region
  # print(region)

  tile_region = ee.Geometry.Polygon(region, None, False)

  # convert "1" to "01"
  tile_number_string = str(tile_number).zfill(2)

  attica_image_base2 = attica_image_base + str(tile_number_string)

  doPrediction(attica_image_base2, user_folder, attica_kernel_buffer, tile_region)

  print('done exporting for tile: ', tile_number_string, region)
  tile_number = tile_number + 1

# #####################################################
# ## FOR SPECIFIC TILES --> (V2) (if failed to run)

# tilesList = [14,29,34,36,45] # if you want to run for specific tiles

# for i in tilesList: # v2 if you want to run for specific tiles

#   tile_number = int(i)

#   # # option a. without buffer
#   # tile = splitted.getInfo()[i-1]
#   # region = tile["coordinates"]
#   # # print(region)

#   # option b. with buffer
#   # buffer the tile 250m to avoid data gaps between tiles
#   # the distance of 250m was found empirically by running the code and seeing the gaps
#   prj = ee.Projection('EPSG:3035');  # European projection
#   buffer = ee.Geometry(ee.List(splitted.get(i-1))).buffer(250,None,prj).bounds(0.1)
#   buffered_tile = buffer.getInfo()
#   buffered_region = buffered_tile["coordinates"]
#   region = buffered_region
#   # print(region)

#   tile_region = ee.Geometry.Polygon(region, None, False)

#   # convert "1" to "01"
#   tile_number_string = str(tile_number).zfill(2)

#   attica_image_base2 = attica_image_base + str(tile_number_string)

#   doPrediction(attica_image_base2, user_folder, attica_kernel_buffer, tile_region)

#   print('done exporting for tile: ', tile_number_string, region)


END OF NOTEBOOK